In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
def foodcsv():
    foodcsv = pd.read_csv('E:/project/main/DRS/DRS/data/food.csv', index_col=0, )
    return foodcsv
def scoredata():
    scorecsv = pd.read_csv('E:/project/main/DRS/DRS/data/score_mod.csv', index_col=0, )
    return scorecsv
def scoredata_original():
    scorecsv = pd.read_csv('E:/project/main/DRS/DRS/data/score.csv', index_col=0, )
    return scorecsv

In [3]:
foodcsv()[["name","type","calories","fat","totalcarbs","sodium","protein","special"]].head(2)

FileNotFoundError: [Errno 2] No such file or directory: 'E:/project/main/DRS/DRS/data/food.csv'

In [ ]:
scoredata().head(2)

In [ ]:
scoredata_original().head(2)

In [ ]:
def euclidean(s1, s2):
    """Take two pd.Series objects and return their euclidean 'similarity'."""
    diff = s1 - s2
    return 1 / (1 + np.sqrt(np.sum(diff ** 2)))

In [ ]:
def cosine(s1, s2):
    """Take two pd.Series objects and return their cosine similarity."""
    return np.sum(s1 * s2) / np.sqrt(np.sum(s1 ** 2) * np.sum(s2 ** 2))

In [ ]:
def pearson(s1, s2):
    """Take two pd.Series objects and return a pearson correlation."""
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

In [ ]:
def knn(user_id,food_id):
    scorecsv_mod = scoredata()
    all_user_profiles = scorecsv_mod.pivot_table('rating', index='food_id', columns='user_id')
    user_condition = scorecsv_mod.user_id != user_id
    food_condition = scorecsv_mod.food_id == food_id
    ratings_by_others = scorecsv_mod.loc[user_condition & food_condition]
   
    if ratings_by_others.empty:
        return pd.DataFrame()

    ratings_by_others.set_index('user_id', inplace=True)
    their_ids = ratings_by_others.index
    their_profiles = all_user_profiles[their_ids]
    user_profile = all_user_profiles[user_id]
    sims = their_profiles.apply(lambda profile: pearson(profile, user_profile), axis=0) 
    sims = sims.to_frame().sort_values(by=0, axis=0, ascending=False)
    sims.rename(columns={0:food_id},inplace=True)
    sims=sims.head(13)
    return sims
def knnf(food_id,user_id):    
    scorecsv = scoredata()
    all_food_profiles = scorecsv.pivot_table('rating', index='user_id', columns='food_id')
    user_condition = scorecsv.user_id == user_id
    food_condition = scorecsv.food_id != food_id
    ratings_by_others = scorecsv.loc[user_condition & food_condition]
   
    if ratings_by_others.empty:
        return pd.DataFrame()
    ratings_by_others.set_index('food_id', inplace=True)
    their_ids = ratings_by_others.index
   
    their_profiles = all_food_profiles[their_ids]
    food_profile = all_food_profiles[food_id]
    sims = their_profiles.apply(lambda profile: pearson(profile, food_profile), axis=0)
    sims = sims.to_frame().sort_values(by=0, axis=0, ascending=False)
    sims=sims.head(13)
    return sims

In [ ]:
knn(68,3).head()

In [ ]:
knnf(1,183).head()

In [ ]:
%timeit -n10 knn(1,1)
%timeit -n10 knnf(1,1)

### Pure KNN Method to predict foods' scores

In [ ]:
class CollabEuclideanReco:
    """ Collaborative filtering using a custom sim(u,u'). """

    def learn(self):
        """ Prepare datastructures for estimation. """
        scorecsv = scoredata()
        self.all_user_profiles = scorecsv.pivot_table('rating', index='food_id', columns='user_id')
        self.all_food_profiles = scorecsv.pivot_table('rating', index='user_id', columns='food_id')

    def estimate(self, user_id, food_id):
        """ Ratings weighted by correlation similarity. """
        scorecsv = scoredata()

        user_condition = scorecsv.user_id != user_id
        food_condition = scorecsv.food_id == food_id
        ratings_by_others = scorecsv.loc[user_condition & food_condition]
        if ratings_by_others.empty:
            return 5.5

        ratings_by_others.set_index('user_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_user_profiles[their_ids]
        user_profile = self.all_user_profiles[user_id]
        sims = their_profiles.apply(lambda profile: pearson(profile, user_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]

        if ratings_sims.empty:
            userest = their_ratings.mean()
        else:
            b = ratings_sims.sort_values(by="sim", axis=0, ascending=False)
            b = b.head(13)
            userest = np.average(b.rating, weights=b.sim)

        user_condition = scorecsv.user_id == user_id
        food_condition = scorecsv.food_id != food_id
        ratings_by_others = scorecsv.loc[user_condition & food_condition]
        if ratings_by_others.empty:
            return 6

        ratings_by_others.set_index('food_id', inplace=True)
        their_ids = ratings_by_others.index
        their_ratings = ratings_by_others.rating
        their_profiles = self.all_food_profiles[their_ids]
        food_profile = self.all_food_profiles[food_id]
        sims = their_profiles.apply(lambda profile: pearson(profile, food_profile), axis=0)
        ratings_sims = pd.DataFrame({'sim': sims, 'rating': their_ratings})
        ratings_sims = ratings_sims[ratings_sims.sim > 0]
        if ratings_sims.empty:
            return (userest + their_ratings.mean()) / 2
        else:
            c = ratings_sims.sort_values(by="sim", axis=0, ascending=False)
            c = c.head(13)
        return (userest + np.average(c.rating, weights=c.sim)) / 2

In [ ]:
rec=CollabEuclideanReco()
rec.learn()
rec.estimate(64,19)

In [ ]:
def knn_array(f,u):
    R = scoredata().pivot_table('rating', index='user_id', columns='food_id').fillna(0)
    a = list(knnf(f,u).index)
    b = list(knn(u,f).index)
    a.append(f)
    b.append(u)
    RC = R.iloc[R.index.isin(b)]
    RT = R.T
    RT = RT.iloc[RT.index.astype(float).isin(a)]
    RO = RC[RT.index]
    R = np.array(RO)
    return R,RO

In [ ]:
R,RO=knn_array(19,64)
RO.head()

In [ ]:
import numpy
def matrix_factorization(u, f, steps=3500, alpha=0.0003, beta=0.02):
    R, RO = knn_array(f, u)
    N = len(R)
    M = len(R[0])
    K = 4
    P = numpy.random.rand(N, K)
    Q = numpy.random.rand(M, K)
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i, :], Q[:, j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P, Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i, :], Q[:, j]), 2)
                    for k in range(K):
                        e += (beta / 2) * (pow(P[i][k], 2) + pow(Q[k][j], 2))
        if e < 0.001:
            break
    Rpred = numpy.dot(P, Q)
    scorepred = pd.DataFrame(Rpred[0:, 0:], index=RO.index, columns=RO.columns)
    try:
        temp=scorepred.loc[u].loc[f]
        if temp>0:
            
            return temp
        else:
            return 0
    except:
        print(u, f, "error")
        return 5.78

In [ ]:
matrix_factorization(64,19)

In [ ]:
realsurvey_64 = pd.read_csv("data/survey/recfood/ID64.csv",index_col=0)
realsurvey_64.loc[19]["score"]

In [ ]:
def healthscore(user, food):
    def dif(a, b):
        return math.fabs(float(math.fabs(a - b) / b))

    ts = 1  
    if food.type == 1:
        ts = 0.6
    elif food.type == 2:
        ts = 0.3
    elif food.type == 3:
        ts = 0.3
    elif food.type == 4:
        ts = 0.2
    elif food.type == 5:
        ts = 0.1
    basic = user.getcalorie() / 3 * ts
    carbs = ts * basic * 0.45
    fatmin = basic * 0.2 / 9.0 * ts
    fatmax = basic * 0.35 / 9.0 * ts
    sodium = 2400 / 3.0 * ts
    if user.age >= 51:
        sodium = 1500 / 3 * ts
    protein = 0.9 * user.getweight()
    if 0.5 <= user.goal <= 1.5:
        protein = 1.5 * user.getweight()
    elif user.goal > 1.5:
        protein = 2.2 * user.getweight()
    if "protein" in user.special:
        protein = user.special["protein"]
    protein /= 3
    protein *= ts
    if food.getfat() in [fatmin, fatmax]:
        fat = 0
    else:
        fat = min(dif(food.getfat(), fatmin), dif(food.getfat(), fatmax))
    diff = dif(food.getcal(), basic) * 0.5 + dif(food.gettotalcarbs(), carbs) * 0.1 + dif(food.getsodium(),
                                                                                          sodium) * 0.1 + dif(
            food.getprotein(), protein) * 0.1 + fat * 0.2
    result -= math.pow(7 * diff, 2) + 100
    if diff <= 0.05:
        result = 100
    if result <= 40: result = 40

    if veg(user, food): result = 0 
    return result

def veg(user, food):
    if "veg" in user.special:
        if food.getspecial("veg"):
            return True
        else:
            return False
def health(p):# input p user_id list
        user = []
        food = fooddata()
        healthcsv = pd.DataFrame()
        for i in p:
            user.append(userselect(i))
        for i in user:
               for j in food:
                    if j.foodid not in healthcsv.columns:
                        healthcsv[j.foodid] = np.nan
                    score = healthscore(i, j)
                    healthcsv.loc[i.id, j.foodid] = score
        healthcsv = healthcsv.round(3)
        return healthcsv

In [ ]:
_health=health([183]).multiply(.1)
_health

In [ ]:
def gene_test_train():
    scorecsv = scoredata()
    scorecsv = scorecsv.ix[np.random.choice(scorecsv.index, size=800, replace=False)]
    user_ids_larger_1 = pd.value_counts(scorecsv.user_id, sort=False) > 1
    user_ids_larger_1 = user_ids_larger_1[user_ids_larger_1].index

    scorecsv = scorecsv.select(lambda l: scorecsv.loc[l, 'user_id'] in user_ids_larger_1)
    assert np.all(scorecsv.user_id.value_counts() > 1)

    def assign_to_set(df):
        sampled_ids = np.random.choice(df.index,
                                       size=np.int64(np.ceil(df.index.size * 0.2)),
                                       replace=False)
        df.ix[sampled_ids, 'for_testing'] = True
        return df

    scorecsv['for_testing'] = False
    grouped = scorecsv.groupby('user_id', group_keys=False).apply(assign_to_set)
    scorecsv_train = scorecsv[grouped.for_testing == False]
    scorecsv_test = scorecsv[grouped.for_testing == True]
    assert len(scorecsv_train.index & scorecsv_test.index) == 0
    scorecsv_train.to_csv('E:/project/main/DRS/DRS/data/test_score/scorecsv_train.csv')
    scorecsv_test.to_csv('E:/project/main/DRS/DRS/data/test_score/scorecsv_test.csv')
gene_test_train()

In [ ]:
scorecsv_train = pd.read_csv('E:/project/main/DRS/DRS/data/test_score/scorecsv_train.csv',index_col=0)
scorecsv_test=pd.read_csv('E:/project/main/DRS/DRS/data/test_score/scorecsv_test.csv',index_col=0)

In [ ]:
scorecsv_train.head()

In [ ]:
scorecsv_test.head()

In [ ]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [ ]:
def evaluate_data():
    train = pd.read_csv("E:/project/main/DRS/DRS/data/test_score/scorecsv_train.csv")
    test = pd.read_csv("E:/project/main/DRS/DRS/data/test_score/scorecsv_test.csv")
    return train, test

def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    scorecsv_train, scorecsv_test = evaluate_data()
    ids_to_estimate = zip(scorecsv_test.user_id, scorecsv_test.food_id)
    estimated = np.array([estimate_f(u, i) for (u, i) in ids_to_estimate])
    real = scorecsv_test.rating.values
    return compute_rmse(estimated, real)

In [ ]:
def naive(a,b):
    return 5.798
evaluate(naive)